In [ ]:
from transformers import PreTrainedTokenizerFast
import torch
import torch.nn as nn
from model_code import GPTModel, GPT_CONFIG_124M, generate


# --------------------------------------
# 1. load the model fom local directory
# --------------------------------------

model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

checkpoint = torch.load('/kaggle/input/sebastian-v4/model_checkpoints/model_pg_190000_steps.pth', weights_only=False)

# modified (added model loading code)
model.load_state_dict(checkpoint["model_state_dict"])


# load the tokenizer
# ----------------------------
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")


# generate a sample
# ----------------------------
prompt = "रामले भात"

generate(
    model,
    prompt,
    tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=None,  # New parameter for nucleus sampling
    eos_id=None,
    repetition_penalty=1.2,
    penalize_len_below=50
)


# ----------------------------
# 2. push model to huggingface
# ----------------------------
# ## import os
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")

model.push_to_hub("Aananda-giri/GPT2-Nepali", token=hf_token)


# ---------------------------------
# reload model from huggingface
# ---------------------------------
from transformers import PreTrainedTokenizerFast
import torch
import torch.nn as nn
from gpt_model import GPTModel, GPT_CONFIG_124M, generate


# load the model
# ----------------------------

model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# load from huggingface
model = GPTModel.from_pretrained("Aananda-giri/GPT2-Nepali")
model.to(device)

# load the tokenizer
# ----------------------------
# tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")


# generate a sample
# ----------------------------

prompt = "रामले भात"

generate(
    model,
    prompt,
    tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=None,  # New parameter for nucleus sampling
    eos_id=None,
    repetition_penalty=1.2,
    penalize_len_below=50
)

In [1]:
# -------------------------------------------------
# 1. Download the model weights (from huggingface)
# -------------------------------------------------
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="Aananda-giri/LLAMA3-Nepali", filename="parameters_300m/model_pg_398000_steps.pth", local_dir="./")


# downloaded to `parameters_300m/model_pg_398000_steps.pth`
!ls parameters_300m/model_pg_398000_steps.pth

model_pg_398000_steps.pth:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

parameters_300m/model_pg_398000_steps.pth


In [29]:
# -------------------
# 2. Load The tokenizer
# -------------------
from transformers import PreTrainedTokenizerFast

# Load the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/LLAMA3-Nepali")
tokenizer.save_pretrained("NepaliBPE")

('NepaliBPE/tokenizer_config.json',
 'NepaliBPE/special_tokens_map.json',
 'NepaliBPE/tokenizer.json')

In [32]:
tokenizer.get_vocab()

{'कार्यसमितिको</w>': 12813,
 'चाउचाउलाई</w>': 29524,
 'लम्बी': 21868,
 'पाठे': 12951,
 'प्रतिष्ठानमा</w>': 10452,
 'कार्यान्वयनमा</w>': 4424,
 'धम्क्याउने</w>': 39313,
 'द्वारा</w>': 2354,
 'पटेल</w>': 20260,
 'ब्राजि': 6414,
 'र्यौली</w>': 28366,
 'लिफ्ट</w>': 19600,
 'डायलाइसिस</w>': 33592,
 'हुन्छन</w>': 30218,
 'सुनौली</w>': 42439,
 'नदेखेपछि</w>': 37528,
 'गोरो</w>': 41763,
 'भरे': 8502,
 'टल': 13108,
 'उकास्न</w>': 21609,
 'पति</w>': 905,
 'सम्झना</w>': 10074,
 'वजन</w>': 31973,
 'उपर': 40121,
 'जित्नैपर्ने</w>': 47952,
 'लुकाउने</w>': 27463,
 'जनक': 3353,
 'लिरहेको</w>': 15937,
 '१४४</w>': 21368,
 '़': 98,
 'बूढानीलकण्ठमा</w>': 40918,
 'श्रमिकको</w>': 10632,
 '९ः३०</w>': 47870,
 'ीलाई</w>': 662,
 'मोहमद</w>': 19745,
 'खोज्दा</w>': 8305,
 'संघका</w>': 3044,
 'हर्षित</w>': 19656,
 'रुची</w>': 20584,
 'यन्स</w>': 34742,
 'पुरस्कृत</w>': 9590,
 'स्वेच्छिक</w>': 28719,
 'हस्पिटलले</w>': 45767,
 'उहाँमा</w>': 30674,
 'अध्ययनलाई</w>': 38083,
 'गर्नसक्ने</w>': 9679,
 'सम्बन्धि': 47430,


In [12]:
# previous_chapters.py

import requests
res=requests.get(r"https://raw.githubusercontent.com/Aananda-giri/LLAMA3-Nepali/main/3.%20training_loop/previous_chapters.py")
with open('previous_chapters.py','w') as f:
    f.write(res.text)

Overwriting previous_chapters.py


In [5]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00


In [21]:
import torch

from previous_chapters import (
    Llama3Model,
    ChatFormat,
    Tokenizer,
    generate_and_print_sample
)

if __name__ == "__main__":
    tokenizer = Tokenizer("NepaliBPE/tokenizer.json")
    chat_tokenizer = ChatFormat(tokenizer)

    print(f'---------------------\nDEBUG MODE=False (300M model)\n---------------------')
    # Llama 3.2 ~300M Scaled Version
    LLAMA32_CONFIG = {
        "vocab_size": 50006,       # <len(tokenizer.tokenizer)=50006> 128_256 reduced vocabulary size
        "context_length": 512,      # 131_072 reduced Context length (unrelated to model size but higheer context length consumes more RAM)
        "emb_dim": 1320,            # 2048 reduced Embedding dimension
        "n_heads": 20,              # 32 reduced Number of attention heads
        "n_layers": 10,             # 16 reduced Number of layers
        "hidden_dim": 5280,         # 8192 Size of the intermediate dimension in FeedForward
        "n_kv_groups": 5,           # 8 Key-Value groups for grouped-query attention
        "rope_base": 500_000.0,     # 500_000 The base in RoPE's "theta"
        "dtype": torch.bfloat16,    # Lower-precision dtype to reduce memory usage
        "rope_freq": {              # RoPE frequency scaling
            "factor": 32.0,
            "low_freq_factor": 1.0,
            "high_freq_factor": 4.0,
            "original_context_length": 8192,
        }
    }

    old_context_length = 131_072    # original context length of llama3.2 model
    new_context_length = LLAMA32_CONFIG["context_length"]  # 512 our new context length

    def rescale_theta(theta_old, context_length_old, context_length_new):
        # original linear scaling
        scaling_factor = context_length_new / context_length_old
        theta_new = theta_old * scaling_factor
        return theta_new

    LLAMA32_CONFIG["rope_base"] = rescale_theta(
        LLAMA32_CONFIG["rope_base"],
        old_context_length,
        new_context_length
    )

    print("New RoPE theta (i.e. LLAMA32_CONFIG[\"rope_base\"]):", LLAMA32_CONFIG["rope_base"])

    model = Llama3Model(LLAMA32_CONFIG)
    # Todo: don't compile
    # compile the model
    if True:
        print("compiling the model... (takes a ~minute)")
        unoptimized_model = model
        model = torch.compile(model) # requires PyTorch 2.0

    # Check buffers
    # --------------
    print('The following is expected to print True to confirm buffers are reused instead of being (wastefully) recreated:')
    print(model.trf_blocks[0].att.mask is model.trf_blocks[-1].att.mask)
    print(model.trf_blocks[0].att.cos is model.trf_blocks[-1].att.cos)
    print(model.trf_blocks[0].att.sin is model.trf_blocks[-1].att.sin)

    # Display number of parameters
    # -----------------------------
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total number of parameters: {total_params:,}")
    # Account for weight tying
    total_params_normalized = total_params - model.tok_emb.weight.numel()
    print(f"\nTotal number of unique parameters: {total_params_normalized:,}")

    # Display model_memory_size
    # -----------------------------------------------------------------------
    def model_memory_size(model, input_dtype=torch.float32):
        total_params = 0
        total_grads = 0
        for param in model.parameters():
            # Calculate total number of elements per parameter
            param_size = param.numel()
            total_params += param_size
            # Check if gradients are stored for this parameter
            if param.requires_grad:
                total_grads += param_size

        # Calculate buffer size (non-parameters that require memory)
        total_buffers = sum(buf.numel() for buf in model.buffers())

        # Size in bytes = (Number of elements) * (Size of each element in bytes)
        # We assume parameters and gradients are stored in the same type as input dtype
        element_size = torch.tensor(0, dtype=input_dtype).element_size()
        total_memory_bytes = (total_params + total_grads + total_buffers) * element_size

        # Convert bytes to gigabytes
        total_memory_gb = total_memory_bytes / (1024**3)

        return total_memory_gb

    print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.2f} GB")
    print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.2f} GB")
    # -----------------------------------------------------------------------

    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    model.to(device)
    print(f'device: {device}')

    latest_model_checkpoint = "parameters_300m/model_pg_398000_steps.pth"

    checkpoint = torch.load(latest_model_checkpoint, weights_only=False)

    # modified (added model loading code)
    model.load_state_dict(checkpoint["model_state_dict"])
    generate_and_print_sample(PROMPT="रामले भात", tokenizer=tokenizer, chat_tokenizer=chat_tokenizer, model=model, device=device, context_length = LLAMA32_CONFIG["context_length"])

---------------------
DEBUG MODE=False (300M model)
---------------------
New RoPE theta (i.e. LLAMA32_CONFIG["rope_base"]): 1953.125
compiling the model... (takes a ~minute)
The following is expected to print True to confirm buffers are reused instead of being (wastefully) recreated:
True
True
True
Total number of parameters: 384,691,560

Total number of unique parameters: 318,683,640
float32 (PyTorch default): 2.87 GB
bfloat16: 1.43 GB
device: cuda
Output text:
 <|start_header_id|>प्रयोगकर्ता <|end_header_id|>

रामले भात <|eot_id|>र अन्य खानेकुरा पनि खान पाएका छैनन् । < | endoftext | > " " " " म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म त यो देशको प्रधानमन्त्री हुँ । म

In [44]:
from previous_chapters3 import generate_and_print_chat

generate_and_print_chat(
    prompt="रामले भात",
    tokenizer=tokenizer,
    chat_tokenizer=chat_tokenizer,
    model=model,
    device=None,
    max_new_tokens=150,
    context_length=None,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2)


Generated text:
 <|start_header_id|>प्रयोगकर्ता <|end_header_id|>

रामले भात <|eot_id|>टाँका लगाएका छन् । यो समाचार आजको गोरखापत्र दैनिकमा छ भने यसमा आज पनि अधिकांश श्रमजीवी पत्रकारको अवस्था सार्वजनिक भएको छैन भन्ने लेखिएको पाइएको समाचारमा जनाइएको थियो तर केही पत्रकारलाई आफूले पालेका कुकुरसँग खाना खाएको र कसैले पिटेको गुनासो आएको भन्दै उनले यसो नगर्न आग्रह गरेका थिए प्रमुख समाचारदाता तीर्थ दाहालले राससमा एक कार्टुन लेखेको भनेर सामाजिक सञ्जालमा पोष्ट गरेपछि सञ्चारकर्मीलाई समेत समाचारको विषयवस्तु बनाएर सचेत गराइएको बताइएको हो उक्त विषयमा पत्रकार तथा सूचना अधिकारी दीपक पाण्डेसँग थाहाखबरका लागि संवाददाता श्याम लामाले गरेको कुराकानी सर्वप्रथम त्यहाँ कार्यरत समाचारदातासित गरिएको अन्तर्वार्ताको भिडियो खिचेर राखिएको रहेछ त्यो किन गरिएन भन्दा मलाई अचम्म लागेको कुरा के लाग्यो मैले यति मात्रै भनें कि मलाइ त खबर नै आउँदैन होला भनी त्यसपछि अर्को रिपोर्टरले तपाईंकै बारेमा अध्ययन गरिरहेको बेला त्यही बेलामा ब्लग राखेको थिएँ कति बजे भयो हामी सबैले सुनेका छौं तीन चार घण्टामा


'<|start_header_id|>प्रयोगकर्ता <|end_header_id|>\n\nरामले भात <|eot_id|>टाँका लगाएका छन् । यो समाचार आजको गोरखापत्र दैनिकमा छ भने यसमा आज पनि अधिकांश श्रमजीवी पत्रकारको अवस्था सार्वजनिक भएको छैन भन्ने लेखिएको पाइएको समाचारमा जनाइएको थियो तर केही पत्रकारलाई आफूले पालेका कुकुरसँग खाना खाएको र कसैले पिटेको गुनासो आएको भन्दै उनले यसो नगर्न आग्रह गरेका थिए प्रमुख समाचारदाता तीर्थ दाहालले राससमा एक कार्टुन लेखेको भनेर सामाजिक सञ्जालमा पोष्ट गरेपछि सञ्चारकर्मीलाई समेत समाचारको विषयवस्तु बनाएर सचेत गराइएको बताइएको हो उक्त विषयमा पत्रकार तथा सूचना अधिकारी दीपक पाण्डेसँग थाहाखबरका लागि संवाददाता श्याम लामाले गरेको कुराकानी सर्वप्रथम त्यहाँ कार्यरत समाचारदातासित गरिएको अन्तर्वार्ताको भिडियो खिचेर राखिएको रहेछ त्यो किन गरिएन भन्दा मलाई अचम्म लागेको कुरा के लाग्यो मैले यति मात्रै भनें कि मलाइ त खबर नै आउँदैन होला भनी त्यसपछि अर्को रिपोर्टरले तपाईंकै बारेमा अध्ययन गरिरहेको बेला त्यही बेलामा ब्लग राखेको थिएँ कति बजे भयो हामी सबैले सुनेका छौं तीन चार घण्टामा'